## Notebook for Calculating Sample Size

This notebook is meant to draft some of the methods to calculate sample size based on Monte-Carlo type simulations.

Importing the necessary libraries:

In [54]:
import math
import numpy as np
from scipy.stats import norm, t, binom
import pandas as pd

### TEST I: Z-test for Comparing a Mean to a Known Value e.g. $\mu = \hat\mu$:

Compare this method to an online sample-size calculator availabel here: https://www.stat.ubc.ca/~rollin/stats/ssize/n1.html


In [2]:
def zSampleSize(muHyp, sigma, test = 'equal', mu = 0, numSim = 2000, maxN = 500, alpha = 0.05, power = 0.8):
    
    """ Compute the sample size for a test to a known mean parameter
    muHyp: the hypothesized mean parameter -user must supply
    sigma: a known population variance -user must supply
    
    test: type of test on parameter eg. one-sided or equal
    mu: the population mean parameter with default 0
    numSim: the number of simulations to be used with default 2000
    maxN: maxium number of samples with default 500
    alpha: the prescribed test size with default value 0.05
    power: the prescribed power of the test with default value 0.8
    
    """
    
    ## check for type of test and get corresponding critical value
    
    if test == 'equal':
        criticalValue =  np.abs(norm.ppf(alpha/2))
    else:
        criticalValue = norm.ppf(1-alpha)
        
    
    ## loop over the 'sample size'
    for i in range(2,maxN):
        
        ## counter to determine the number of times the test statistic exceeds the critical value
        rejectedCounter = 0
        
        ## create simulations based on user input
        for j in range(numSim):
            
            ## simulate a normal random variable and compute z-statistic
            simulated = np.random.normal(muHyp, sigma, i)
            zStatistic = (simulated.mean() - mu) / (sigma / np.sqrt(i))
            
            ## check if the statistic exceeds the specified critical value
            if  test == 'equal':                
                if np.abs(zStatistic) > criticalValue:
                    rejectedCounter += 1
            else:
                if np.sign(muHyp - mu) > 0:
                    if zStatistic > criticalValue:
                        rejectedCounter += 1
                else:
                    if zStatistic < -criticalValue:
                        rejectedCounter += 1

        ## check if we've achived sufficent power
        if rejectedCounter/numSim >= power:
            return(i)
            break
 

In [4]:
zSampleSize(2,3,'equal',3)

69

### Test II: T-Test Comparing Two Independent Samples $\mu_1 \mbox{vs} \mu_2$

Here we assume that the sample sizes from the two samples is the same. We specify the population variance and assume both samples have the same $\sigma$, but it will be estimated from the sample invoking the indpendent samples t-test

https://www.stat.ubc.ca/~rollin/stats/ssize/n2.html

In [5]:
def tSampleSize(mu1, mu2, sigma, test = 'equal', numSim = 2000, maxN = 500, alpha = 0.05, power = 0.8):
    """ Compute the sample size for a indpendent sample t-test
    mu1: the hypothesized mean parameter for group 1 -user must supply
    mu2: the hypothesized mean parameter for group 2 -user must supply
    sigma: a known population variance -user must supply
    
    test: type of test on parameter eg. one-sided or equal
    mu: the population mean parameter with default 0
    numSim: the number of simulations to be used with default 2000
    maxN: maxium number of samples with default 500
    alpha: the prescribed test size with default value 0.05
    power: the prescribed power of the test with default value 0.8 
    """
    
    ## loop over the 'sample size'
    for i in range(3, maxN):
        
        ## counter to determine the number of times the test statistic exceeds the critical value
        rejectedCounter = 0
        
            ## check for type of test and get corresponding critical value
    
        if test == 'equal':
            criticalValue =  np.abs(t.ppf(alpha/2, 2*i - 2))
        else:
            criticalValue = np.abs(t.ppf(alpha, 2*i - 2))
        
        ## create simulations based on user input
        for j in range(numSim):
            
            ## simulate from two normal distributions
            simulated1 = np.random.normal(mu1, sigma, i)
            simulated2 = np.random.normal(mu2, sigma, i)
            
            ## calculate summaries for test statistic
            muHat1 = simulated1.mean()
            muHat2 = simulated2.mean()
            S1= simulated1.var()
            S2 = simulated2.var()
            varPooled = (S1 + S2)/2
            
            ## calculate test statistic
            tStatistic = (muHat1 - muHat2) / np.sqrt(varPooled * (2/i))
            
            ## check if the t-statistic exceeds the critival value and increment counter
            if  test == 'equal':                
                if np.abs(tStatistic) > criticalValue:
                    rejectedCounter += 1
            else:
                if np.sign(mu1 - mu2) > 0:
                    if tStatistic > criticalValue:
                        rejectedCounter += 1
                else:
                    if tStatistic < -criticalValue:
                        rejectedCounter += 1
                        
        ## check if we've achived sufficent power
        if rejectedCounter/numSim > power:
            return(i)
            break
    
    

In [6]:
tSampleSize(2, 3, 2, 'one-sided')

51

### Test III: Comparing a Proportion to a Known Value $p = \hat p$

We test a difference in proportion from a hypothesized proprtion from a known propoportion. We simulate not only the number of times we achive the apporpriate power but also simulate sampling from and infinite population to compute the sample proportion. The continuity correction is applied to the z-test statistic. 

https://www.stat.ubc.ca/~rollin/stats/ssize/b1.html

In [78]:
def PropSampleSize(pHyp, pKnw, test = 'equal', numSim = 2000, maxN = 500, alpha = 0.05, power = 0.8):
    
    """ Compute the sample size for a test to a known mean parameter
    pHyp: the hypothesized proportion value -user must supply
    pKnw: the population proportion parameter - user must supply
    
    test: type of test on parameter eg. one-sided or equal
    numSim: the number of simulations to be used with default 2000
    maxN: maxium number of samples with default 500
    alpha: the prescribed test size with default value 0.05
    power: the prescribed power of the test with default value 0.8
    
    """
    
     ## check for type of test and get corresponding critical value
    
    if test == 'equal':
        criticalValue =  np.abs(norm.ppf(alpha/2))
    else:
        criticalValue = norm.ppf(1-alpha)   
    
    ## loop over the 'sample size'
    for i in range(2,maxN):
        
        ## counter to determine the number of times the test statistic exceeds the critical value
        rejectedCounter = 0
        
        ## create simulations based on user input
        for j in range(numSim):
            
            ## simulate a normal random variable and compute z-statistic
            simulated = np.random.binomial(1, pHyp, i)
            phat = simulated.mean()
            
            ## get value for continuity correction
            
            if pKnw > phat:
                c = 1 / (2*i)
            elif pKnw < phat: 
                c = - 1 / (2*i)
            elif np.abs(pKnw - phat) < 1 / (2*i):
                c = 0
            
            zStatistic = ((phat - pKnw) + c) / np.sqrt((pKnw * (1 - pKnw)) / i)
            
            ## check if the statistic exceeds the specified critical value
            if  test == 'equal':                
                if np.abs(zStatistic) > criticalValue:
                    rejectedCounter += 1
            else:
                if np.sign(phat - pKnw) > 0:
                    if zStatistic > criticalValue:
                        rejectedCounter += 1
                else:
                    if zStatistic < -criticalValue:
                        rejectedCounter += 1

        ## check if we've achived sufficent power
        if rejectedCounter/numSim >= power:
            return(i)
            break
 

In [88]:
PropSampleSize(0.25, 0.5)

30

### Test IIV: Comparing two Proportion $\hat p_1 = \hat p_2$

We test a difference in proportions between two sampled proportions. We simulate not only the number of times we achive the apporpriate power but also simulate sampling from and infinite population to compute the sample proportions. The continuity correction is applied to the z-test statistic. 

https://www.stat.ubc.ca/~rollin/stats/ssize/b2.html

In [129]:
def TwoPropSampleSize(p1, p2, test = 'equal', numSim = 2000, maxN = 500, alpha = 0.05, power = 0.8): 
    """ Compute the sample size for a test to a known mean parameter
    pHyp: the hypothesized proportion value -user must supply
    pKnw: the population proportion parameter - user must supply
    
    test: type of test on parameter eg. one-sided or equal
    numSim: the number of simulations to be used with default 2000
    maxN: maxium number of samples with default 500
    alpha: the prescribed test size with default value 0.05
    power: the prescribed power of the test with default value 0.8
    
    """
    
     ## check for type of test and get corresponding critical value
    
    if test == 'equal':
        criticalValue =  np.abs(norm.ppf(alpha/2))
    else:
        criticalValue = np.abs(norm.ppf(alpha))   
    
    ## loop over the 'sample size'
    for i in range(2,maxN):
        
        ## counter to determine the number of times the test statistic exceeds the critical value
        rejectedCounter = 0
        
        ## create simulations based on user input
        for j in range(numSim):
            
            ## simulate a normal random variable and compute z-statistic
            simulated1 = np.random.binomial(1, p1, i)
            simulated2 = np.random.binomial(1, p2, i)
            phat1 = simulated1.mean()
            phat2 = simulated2.mean()
            
            ## helper terms
            pnet = np.concatenate([simulated1, simulated2]).mean() ####(simulated1.sum() + simulated2.sum())/ (2*i)
            denom = np.sqrt(np.abs(pnet*(1-pnet)*2/i))
            
            zStatistic = (phat1 - phat2)  / denom
            
            ## check if the statistic exceeds the specified critical value
            if  test == 'equal':                
                if np.abs(zStatistic) > criticalValue:
                    rejectedCounter += 1
            else:
                if np.sign(phat - pKnw) > 0:
                    if zStatistic > criticalValue:
                        rejectedCounter += 1
                else:
                    if zStatistic < -criticalValue:
                        rejectedCounter += 1

        ## check if we've achived sufficent power
        if rejectedCounter/numSim >= power:
            return(i)
            break
 

In [130]:
TwoPropSampleSize(0.5, 0.75)

C:\Users\jddru\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


58

In [124]:
simulated1 = np.random.binomial(1, 0.5, 5)
simulated2 = np.random.binomial(1, 0.5, 5)

In [125]:
np.concatenate([simulated1, simulated2]).mean()

0.5

In [116]:
type(simulated1)

numpy.ndarray